<a href="https://colab.research.google.com/github/RahmankhanA/CNN-Model-Optimization-Using-Keras-Tuner/blob/main/CNN_Model_Optimization_usin_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***CNN Model Optimization usin Keras Tuner***

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [ ]:
print(tf.__version__)

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_image, train_label),(test_image,test_label)=fashion_mnist.load_data()

In [ ]:
train_image=train_image/255.0
test_image=test_image/255.0

In [ ]:
train_image[9].shape


In [ ]:
train_image=train_image.reshape(len(train_image),28,28,1)
test_image=test_image.reshape(len(test_image),28,28,1)

In [ ]:
train_image[9].shape

In [ ]:
def build_model(hp):
  model= keras.Sequential([
          tf.keras.layers.Conv2D(
              filters=hp.Int('conv_1_filter',min_value=64, max_value=128, step=16),
              kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
              activation='relu',
              input_shape=(28, 28, 1)
          ),  
          tf.keras.layers.Conv2D(
              filters=hp.Int('conv_2_filter',min_value=32, max_value=64, step=16),
              kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
              activation='relu',
             
          ),   

          tf.keras.layers.Flatten(),   
          tf.keras.layers.Dense(
              units=hp.Int('dense_1_units',min_value=32, max_value=128, step=16),
              activation='relu',
          ),
          tf.keras.layers.Dense(10, activation='softmax')



  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )

  return model


In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy', 
                          max_trials=5, directory='output', 
                          project_name='Mnist Fashion')

In [ ]:
tuner_search.search(train_image,train_label, epochs=4, validation_split=0.1)

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(train_image,train_label, epochs=15, validation_split=0.1, initial_epoch=4)